In [1]:
from statsmodels.stats.meta_analysis import combine_effects
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import warnings

In [2]:
os.chdir('/project/ssverma_shared/projects/Endometriosis/Endo_Subtyping_Heterogeneity/Meta_Analysis/')

In [3]:
! snakemake -n all --quiet

Building DAG of jobs...
Job stats:
job                        count    min threads    max threads
-----------------------  -------  -------------  -------------
align_to_PMBB                  4              1              1
all                            1              1              1
get_all_SNPs                   5              1              1
lift_snplist_b37_to_b38        2              1              1
total                         12              1              1



In [4]:
phenos = ['endometriosis',
          'cluster_vs_controls_1',
          'cluster_vs_controls_2',
          'cluster_vs_controls_3',
          'cluster_vs_controls_4',
          'cluster_vs_controls_5']

In [7]:
pheno_ss = {p: [] for p in phenos}
pheno_ss_AFR_EUR = {p: [] for p in phenos}
pheno_ss_EUR = {p: [] for p in phenos}
pheno_ss_AFR = {p: [] for p in phenos}

# biobanks = ['eMERGE', 'UKBB', 'PMBB', 'AOU']
biobanks = ['eMERGE', 'UKBB', 'PMBB', 'AOU', 'BioVU']
# biobanks = ['eMERGE', 'PMBB', 'AOU']

for d in biobanks:
    for a in ['AFR', 'EUR', 'ASIAN']:
        if d in ['eMERGE', 'BioVU'] and a == 'ASIAN':
            continue
        for f in os.listdir(f'{d}_{a}/Sumstats/'):
            file_pheno = f.split('.')[0]
            if '0' in file_pheno or 'nc' in file_pheno:
                continue
            pheno_ss[file_pheno].append(f'{d}_{a}/Sumstats/{f}')
            if a != 'ASIAN':
                pheno_ss_AFR_EUR[file_pheno].append(f'{d}_{a}/Sumstats/{f}')
            if a == 'EUR':
                pheno_ss_EUR[file_pheno].append(f'{d}_{a}/Sumstats/{f}')
            if a == 'AFR':
                pheno_ss_AFR[file_pheno].append(f'{d}_{a}/Sumstats/{f}')

In [8]:
eMERGE_id_map = pd.read_table('SNP_Lists/eMERGE_to_PMBB_ID_map.txt', index_col=['ID'])['PMBB_ID']
UKBB_id_map = pd.read_table('SNP_Lists/UKBB_to_PMBB_ID_map.txt', index_col=['ID'])['PMBB_ID']
AOU_id_map = pd.read_table('SNP_Lists/AOU_to_PMBB_ID_map.txt', index_col=['ID'])['PMBB_ID']
BioVU_id_map = pd.read_table('SNP_Lists/BioVU_to_PMBB_ID_map.txt', index_col=['ID'])['PMBB_ID']

In [9]:
use_allele = pd.read_table('PMBB_AFR/Sumstats/endometriosis.saige.gz', index_col='ID')['Allele2']
use_allele

ID
chr1:21686033:C:T     T
chr1:21720782:C:T     T
chr1:21722171:C:T     T
chr1:21772447:C:T     T
chr1:21772448:C:T     T
                     ..
chr9:133370796:C:A    A
chr9:133372523:G:C    C
chr9:133402020:C:T    T
chr9:133405414:T:G    G
chr9:133405868:C:T    T
Name: Allele2, Length: 11425, dtype: object

In [11]:
os.makedirs('Meta_Input/', exist_ok=True)

for p in phenos:
    print(p)
    effs = pd.DataFrame()
    errs = pd.DataFrame()
    alleles = pd.DataFrame()
    afs = pd.DataFrame()
    Ns = pd.DataFrame()
    Neffs = pd.DataFrame()
    cases = pd.DataFrame()
    controls = pd.DataFrame()

    # Load data
    for ss_file in pheno_ss[p]:
        ss_dataset = ss_file.split('_')[0]
        ss_cohort = ss_file.split('/')[0]
        temp = pd.read_table(ss_file, nrows=None)
        temp = temp.rename(columns={'MarkerID': 'ID', 'p.value': 'P'})
        temp = temp.set_index('ID')
        temp['N'] = temp[['N_case', 'N_ctrl']].sum(axis=1)
        temp['Neff'] = temp['N'] * (temp['N_case'] / temp['N']) * (temp['N_ctrl'] / temp['N'])
        temp = temp[temp['AF_Allele2'] >= 0.01]
        temp = temp[['BETA', 'Allele2', 'SE', 'AF_Allele2', 'N', 'Neff', 'N_case', 'N_ctrl']].dropna()

        if ss_dataset == 'eMERGE':
            temp = temp[temp.index.isin(eMERGE_id_map.index)]
            temp = temp.rename(index=eMERGE_id_map)
        if ss_dataset == 'UKBB':
            temp = temp[temp.index.isin(UKBB_id_map.index)]
            temp = temp.rename(index=UKBB_id_map)
        if ss_dataset == 'AOU':
            temp = temp[temp.index.isin(AOU_id_map.index)]
            temp = temp.rename(index=AOU_id_map)
        if ss_dataset == 'BioVU':
            temp = temp[temp.index.isin(BioVU_id_map.index)]
            temp = temp.rename(index=BioVU_id_map)

        effs[ss_cohort] = temp['BETA']
        errs[ss_cohort] = temp['SE']
        alleles[ss_cohort] = temp['Allele2']
        afs[ss_cohort] = temp['AF_Allele2']
        Ns[ss_cohort] = temp['N']
        Neffs[ss_cohort] = temp['Neff']
        cases[ss_cohort] = temp['N_case']
        controls[ss_cohort] = temp['N_ctrl']

    keep_vars = effs.index.intersection(use_allele.index)
    effs, errs, alleles  = effs.loc[keep_vars], errs.loc[keep_vars], alleles.loc[keep_vars]
    afs, Ns, Neffs = afs.loc[keep_vars], Ns.loc[keep_vars], Neffs.loc[keep_vars]
    cases, controls = cases.loc[keep_vars], controls.loc[keep_vars]

    # Now we have data to subset for meta-analysis
    effs = effs[effs.count(axis=1) > 1]
    errs = errs.loc[effs.index]
    alleles = alleles.loc[effs.index]
    afs = afs.loc[effs.index]
    Ns = Ns.loc[effs.index]
    Neffs = Neffs.loc[effs.index]
    cases = cases.loc[effs.index]
    controls = controls.loc[effs.index]

    use_allele_temp = use_allele.loc[alleles.index]
    use_allele_2D = np.broadcast_to(use_allele_temp.T, alleles.T.shape).T
    flip_coords = np.where(alleles.values != use_allele_2D)
    effs.values[flip_coords] = -effs.values[flip_coords]
    afs.values[flip_coords] = 1-afs.values[flip_coords]

    for cohort, effect_col in effs.items():
        table = pd.concat([effect_col, errs[cohort], use_allele], axis=1)
        table.columns = ['BETA', 'SE', 'A1']
        table.index.name = 'SNP'
        table['OR'] = np.exp(table['BETA'])
        table = table.dropna().reset_index()
        table[['CHR', 'BP']] = table['SNP'].str.replace('chr', '').str.split(':', expand=True)[[0, 1]]
        need_cols = ['CHR', 'BP', 'SNP', 'OR', 'SE', 'A1']
        table[need_cols].to_csv(f'Meta_Input/{cohort}.{p}.txt', sep='\t', index=False)

    # Use different combinations of groups
    for group, ss_dict in zip(['ALL', 'AFR_EUR', 'EUR', 'AFR'], [pheno_ss, pheno_ss_AFR_EUR, pheno_ss_EUR, pheno_ss_AFR]):
        print('\t', group)
        ss_group = ss_dict[p]
        cohorts = [f.split('/')[0] for f in ss_group]

        allele_counts = afs[cohorts] * Ns[cohorts]
        meta_af = allele_counts.sum(axis=1) / Ns.sum(axis=1)
        meta_var_info = pd.concat([meta_af, Ns.sum(axis=1), Neffs.sum(axis=1), cases.sum(axis=1), controls.sum(axis=1)], axis=1)
        meta_var_info.columns = ['Meta_AF', 'N', 'Neff_sum', 'Cases_sum', 'Controls_sum']
        meta_var_info['Phi'] = meta_var_info['Cases_sum'] / meta_var_info['N']
        meta_var_info['Neff_meta'] = meta_var_info['N'] * meta_var_info['Phi'] * (1-meta_var_info['Phi'])
        print(meta_var_info)
        meta_var_info.to_csv(f'Meta_Output/{p}.{group}.variant_info')

        if len(cohorts) < 2:
            continue

        plink_cmd_parts = ['plink', '--meta-analysis']
        plink_cmd_parts.extend([f'Meta_Input/{cohort}.{p}.txt' for cohort in cohorts])
        plink_cmd_parts.extend(['--out', f'Meta_Output/{p}.{group}.PLINK'])
        cmd = ' '.join(plink_cmd_parts)

        ! module load plink/1.90Beta6.18; {cmd}



endometriosis
	 ALL
                    Meta_AF         N      Neff_sum  Cases_sum  Controls_sum  \
ID                                                                             
chr1:21686033:C:T  0.147609  447268.0  11336.722013    11724.0      435544.0   
chr1:21720782:C:T  0.149142  447268.0  11336.722013    11724.0      435544.0   
chr1:21722171:C:T  0.150214  433998.0  10838.942359    11206.0      422792.0   
chr1:21772447:C:T  0.148428  447268.0  11336.722013    11724.0      435544.0   
chr1:21772448:C:T  0.008162   79200.0   3311.068933     3489.0       75711.0   
...                     ...       ...           ...        ...           ...   
chr19:8697017:A:T  0.386220  447268.0  11336.722013    11724.0      435544.0   
chr19:8702698:C:T  0.384135  447268.0  11336.722013    11724.0      435544.0   
chr19:8703043:G:A  0.384038  447268.0  11336.722013    11724.0      435544.0   
chr19:8703125:G:T  0.698780  447268.0  11336.722013    11724.0      435544.0   
chr19:8704019:C:T  0